In [13]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [4]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


In [5]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass
    
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
#news_df = pd.DataFrame({'date':news_dates,'title':news_titles})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:미세먼지

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):20

크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=미세먼지&start=91', 'https://search.naver.com

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:12<00:00,  3.00it/s]


검색된 기사 갯수: 총  200 개

[뉴스 제목]
['태국법원 "치앙마이 최악 미세먼지 사태, 정부관리 소홀 책임도"', '아라온, 공공차량 매연ㆍ미세먼지 저감장치로 탄소중립 실현', "[오늘 날씨] 폭우에 무더위에 열대야까지… 미세먼지는 '보통'", '황사철 백령도보다 심각…초미세먼지 기준 초과한 지하철은', '지하철역 미세먼지 농도 감소 추세', "김해 3개 초등교 통학로에 미세먼지·교통사고 막는 '그린숲'", "산불로 숨막혔던 美, 독립기념일 '폭죽 미세먼지' 우려", '예천군, 경북 최초로 시내버스에 미세먼지 저감 필터 부착', '서울 지하철역 초미세먼지 측정했더니…21%가 법적 기준 초과', "경희대 연구팀, 미세먼지 주범 '질소산화물' 제거 촉매 개발", "충남권 한때 비, 최고 체감 온도 33도 이상…미세먼지 '좋음'", '강원고성군, 2023년 하반기 미세먼지 불법배출 민간감시원 운영', "[오늘 날씨] '폭염 기승'… 낮 최고 34도·미세먼지 '보통'", '생기연, 미세먼지·악취 동시에 제거한다…울산서 실증 착수', "성주군 '쿨링로드'로 도로 온도 낮추고, 미세먼지도 잡는다", '[오늘의 날씨] 경기(6일, 목)…습하고 무더워, 미세먼지 ‘보통’', '인재 양성부터 미세먼지 저감까지…KB국민은행의 사회공헌', '미세먼지·악취 동시에 잡는다', "[오늘 날씨] 강한 장맛비, 최대 250㎜… 낮 최고 30도·미세먼지 '맑음'", "현대엔지니어링 '힐스테이트 뉴포레', 도림천 흐르는 역세권…미세먼지 차단 설비로 상쾌한 생활", '롯데홈쇼핑, 풍납초에 `미세먼지 저감 숲` 조성', "이번 주말 수도권 지역, 낮 최고 35도...미세먼지 '보통'", '"초미세먼지 해결하자"…한중 환경장관, 협력계획 갱신 체결', '초미세먼지에 붙어서 독성 일으키는 세포 찾았다', "캐다다 최악 산불로 美 17개주에 대기질 경보…미세먼지 '심각'", "영주시, 삼각지 폐선부지 '미세먼지 차단숲' 조성", '\n\t\t\t미세먼지 모르는 배현성, 진짜 과거에서 타임슬립

중복 제거 후 행 개수:  39


In [6]:
news_df

,date,title,link,content
0,2023-07-11 13:13:56,"태국법원 ""치앙마이 최악 미세먼지 사태, 정부관리 소홀 책임도""",https://n.news.naver.com/mnews/article/001/001...,[\n올해 3∼4월 대기오염 세계 최악 수준…현지 주민 집단소송\n\n\n\n태국 ...
1,2023-07-11 14:36:03,"아라온, 공공차량 매연ㆍ미세먼지 저감장치로 탄소중립 실현",https://n.news.naver.com/mnews/article/011/000...,"[\n에코맥스 플러스, 공공기관 중심으로 빠르게 확산\n\n\n\n[서울경제] 조달..."
2,2023-07-09 04:47:00,[오늘 날씨] 폭우에 무더위에 열대야까지… 미세먼지는 '보통',https://n.news.naver.com/mnews/article/417/000...,[\n\n\n\n\n일요일인 9일은 전국에 비가 쏟아진 뒤 무더위와 열대야가 이어질...
3,2023-07-06 14:33:37,황사철 백령도보다 심각…초미세먼지 기준 초과한 지하철은,https://n.news.naver.com/mnews/article/025/000...,"[\n \n\n\n\n지난 2월 서울에 초미세먼지 주의보가 발령된 날, 서울 중구..."
4,2023-07-06 09:49:14,지하철역 미세먼지 농도 감소 추세,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[서울=뉴시스] 김선웅 기자 = 6일 서울의 한 지하철역에서 공...
5,2023-07-11 09:37:09,김해 3개 초등교 통학로에 미세먼지·교통사고 막는 '그린숲',https://n.news.naver.com/mnews/article/001/001...,"[\n김해시, 2021년부터 8개 초등학교 통학로에 숲 조성\n\n\n\n김해 푸른..."
6,2023-07-04 10:40:56,"산불로 숨막혔던 美, 독립기념일 '폭죽 미세먼지' 우려",https://n.news.naver.com/mnews/article/448/000...,[\n\n\n\n\n/AP=연합뉴스캐나다 산불로 인한 미세먼지를 경고했던 미국 환경...
7,2023-07-06 13:55:01,"예천군, 경북 최초로 시내버스에 미세먼지 저감 필터 부착",https://n.news.naver.com/mnews/article/088/000...,[\n버스 1대당 연간 나무 58그루 미세먼지 저감 효과...총 7대 부착\n\n\...
8,2023-07-06 06:32:00,서울 지하철역 초미세먼지 측정했더니…21%가 법적 기준 초과,https://n.news.naver.com/mnews/article/421/000...,[\n서울시의회 3월31일~4월10일 출근시간대 측정 결과1호선이 평균 83.1㎍/...
9,2023-07-04 22:05:05,"경희대 연구팀, 미세먼지 주범 '질소산화물' 제거 촉매 개발",https://n.news.naver.com/mnews/article/003/001...,[\n상용화된 촉매 대비 성능 3~4배 향상\n\n\n\n[용인=뉴시스] 경희대 화...
